# Decorators

### 1- Function can received function as input

In [ ]:
def sum(a, b):
    return a + b

def sub(a, b):
    return a - b

def apply(funct, a, b):
    return funct(a,b)

apply(sum, 2, 3)

### 2- Function can return function as output

In [ ]:
def power(n):
    def func(number):
        return number**n
    return func

pow2 = power(2)
pow3 = power(3)

pow2(3), pow3(3)
    

### 3- Behavior : Combine of two 

In [ ]:
import time
import random

def stopwatch(f):
    def func(*args, **kwargs):
        tic = time.time()
        result = f(*args, **kwargs)
        print(f'this function took: {time.time() - tic}')
        return result
    return func



### 4- Decorator

In [ ]:
@stopwatch
def sleep_random(s=1):
    time.sleep(s + random.random()/100)
    return 'Done !'
sleep_random(1), sleep_random(2), sleep_random(3)


### 5-Wraps

In [ ]:
import time
import random
from functools import wraps 


def stopwatch(f):
    @wraps(f)
    def func(*args, **kwargs):
        tic = time.time()
        result = f(*args, **kwargs)
        print(f"this function took: {time.time() - tic}")
        return result
    return func

@stopwatch
def sleep_random(s):
    """This function sleeps at least for `s` seconds."""
    return time.sleep(s + random.random())

timed_sleep = stopwatch(sleep_random)

### 6- Stacks

In [ ]:
import time
import random
from functools import wraps 


def print_call1(f):
    @wraps(f)
    def func(*args, **kwargs):
        result = f(*args, **kwargs)
        print(f"print call arguments 1: {args}")
        return result
    return func

def print_call2(f):
    @wraps(f)
    def func(*args, **kwargs):
        result = f(*args, **kwargs)
        print(f"print call arguments 2: {args}")
        return result
    return func

@print_call1
@print_call2
def sleep_random(s):
    """This function sleeps at least for `s` seconds."""
    return time.sleep(s + random.random())

sleep_random(1.5)

### 7- inputs

In [ ]:
import time
import random
from functools import wraps 


def loggg(show_time=True, show_name=True):
    def stopwatch(f):
        @wraps(f)
        def func(*args, **kwargs):
            tic= time.time()
            result = f(*args, **kwargs)
            log_text = 'call'
            if show_name:
                log_text = f"{log_text} name: {f.__name__}"
            if show_time:
                log_text = f"{log_text} name: {f.__name__}"
            print(log_text)
            return result
        return func
    return stopwatch

@loggg(show_time=False, show_name=True)
def sleep_random(s):
    """ This function sleeps at least 's' seconds. """
    time.sleep(s + random.random()/100)
    return "Done!"

sleep_random(1)
            

### 7- Optional inputs

In [ ]:
import time
import random
from functools import wraps 


def loggg(func_in=None,*,show_time=True, show_name=True):
    def stopwatch(f):
        @wraps(f)
        def func(*args, **kwargs):
            tic= time.time()
            result = f(*args, **kwargs)
            log_text = 'call'
            if show_name:
                log_text = f"{log_text} name: {f.__name__}"
            if show_time:
                log_text = f"{log_text} name: {f.__name__}"
            print(log_text)
            return result
        return func
    if func_in is None:
        return stopwatch
    else:
        return stopwatch(func_in)

@loggg(show_time=False, show_name=True)
def sleep_random(s):
    """ This function sleeps at least 's' seconds. """
    time.sleep(s + random.random()/100)
    return "Done!"

sleep_random(1)

### 8- Cas pratiques : pandas-logger decorator

In [ ]:
from functools import wraps
import datetime as dt

def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"just ran step {func.__name__} shape={result.shape} took {time_taken}s")
        return result
    return wrapper

In [ ]:
import pandas as pd

df = pd.read_csv('../../bigmac.csv')

@log_step
def start_pipeline(dataf):
    return dataf.copy()

@log_step
def set_dtypes(dataf):
    return (dataf
            .assign(date=lambda d: pd.to_datetime(d['date']))
            .sort_values(['currency_code', 'date']))

@log_step
def remove_outliers(dataf, min_row_country=32):
    countries = (dataf
                .groupby('currency_code')
                .agg(n=('name', 'count'))
                .loc[lambda d: d['n'] >= min_row_country]
                .index)
    return (dataf
            .loc[lambda d: d['currency_code'].isin(countries)])

df_new = (df
  .pipe(start_pipeline)
  .pipe(set_dtypes)
  .pipe(remove_outliers, min_row_country=20))